# Graphical Error Modelling

This notebook details about algorithms discussed in section 3 of the paper, "Alignment Analysis of Sequential Segmentation of Lexicons to Improve Automatic Cognate Detection"

## Imports

The function `common_elements` will carry out the operation of set intersection and `uncommon_elements` will carry out the operation of set difference.

In [1]:
from math import ceil, floor

def common_elements(list1, list2):
    ''' Carries out set intersection '''
    return [element for element in list1 if element in list2]

def uncommon_elements(list1, list2):
    ''' Carries out set difference '''
    return [element for element in list1 if element not in list2]

We import the functions from the previous notebook which are `shingle` and `two_ends`.

In [5]:
def shingle(input, k):
    ''' Shingles the input into k-grams '''
    k = min(len(input), k)
    start_combinations = [input[:i] for i in range(1, k)]
    kgrams = [input[i:i + k] for i in range(len(input) - k + 1)]
    end_combinations = [input[-i:] for i in range(k - 1, 0, -1)]
    return start_combinations + kgrams + end_combinations

def two_ends(input, k):
    ''' Shingles the input into k-grams but encodes numbers from two ends '''
    basic = shingle(input, k)
    result =[]
    for i in range(1, len(basic) + 1):
        if i <= (len(input) - i + 2):
            result.append(str(i) + basic[i - 1]) # Append numbers from start
        else:
            result.append(basic[i - 1] + str(len(basic) - i + 1)) # Append numbers from end
    return result

## Graphical Modelling Algorithm

The algorithm consists of three parts:
1. Initialization
2. Equalization of the set cardinalities
3. Inserting the mappings of the set members into the graph

This algorithm returns the graphical structure between two shingle sets.

In [3]:
def graph_model(first, second):
    ''' Constructs the graphical structure between two shingle sets. '''
    
    # Step 1: Initialization
    # If the given sets first and second are empty, we initialize 
    # them by inserting an empty token, (nun), into those sets.
    
    if len(first) == 0:
        first.append("nun") #insert empty token if found empty
    if len(second) == 0:
        second.append("nun") #insert empty token if found empty
        
    # Step 2: Equalization of the set cardinalities
    # The cardinalities of the sets first and second made
    # equal by inserting empty tokens (nun) into the
    # middle of the sets.
    
    # While loops to equalize the sizes
    while(len(first) < len(second)):
        pos = ceil(len(first) / 2)
        first.insert(pos, "nun")
    
    # While loops to equalize the sizes
    while(len(first) > len(second)):
        pos = floor(len(second) / 2)
        second.insert(pos, "nun")
        
    # Step 3: Inserting the mappings of the set members into the graph
    # The empty graph is initialized as graph = {}.
    # The directed edges are generated, originating from every set member
    # of first to every set member of second. This results in a complete 
    # directed bipartite graph between first and second sets.
    
    # Pairs in tuples
    graph = set() #Graph in sets to avoid duplicates
    
    for i in range(len(first)):
        pair = (first[i], second[i]) # One to one mapping with same index
        graph.add(pair)
    for i in range(len(first) - 1):
        pair = (first[i], second[i + 1]) # One to one mapping with an index ahead
        graph.add(pair)
    if len(first) > 1:
        for i in range(1, len(first)):
            pair = (first[i], second[i - 1]) # One to one mapping with an index before
            graph.add(pair)
    return graph

## Playing with the functions 

Let the source cognate be *mesia* and target cognate be *messia*.

Firstly, we will shingle them using `two_ends` function.

Using the `common_elements` function, we would find $S \cap T$.

Using the `uncommon_elements` function, we would find $S - (S \cap T)$, which would form the *top*.

Using the `uncommon_elements` function, we would find $T - (S \cap T)$, which would form the *bottom*.

Using the `graph_model` function, the graph would be outputted.

In [9]:
source = two_ends("mesia", 2) # Source cognate
target = two_ends("messia", 2) # Target cognate
print(source, target)
st = common_elements(source, target) # s cap t
top = uncommon_elements(source, st) # s - (s cap t)
bottom = uncommon_elements(target, st) # t - (s cap t)
print("Top and bottom are", top, bottom)
print("Graph is ", graph_model(top, bottom))

['1m', '2me', '3es', 'si3', 'ia2', 'a1'] ['1m', '2me', '3es', '4ss', 'si3', 'ia2', 'a1']
Top and bottom are [] ['4ss']
Graph is  {('nun', '4ss')}


We get the graph as `{('nun', '4ss')}`, which means as $\phi \rightarrow 4ss$.
Intuitively, $\phi \rightarrow 4ss$ means that if the letter s is added at position 4 of the word of the source *mesia*, then one could get the target word *messia*.